In [2]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import json
import git


In [3]:
# get commit number for github repository
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
print(sha)

edc918595761fee80a305087184f580b699b21b3


In [4]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 10, 
}




In [6]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : "https://github.com/mapol-chem/qed-ci",
      "branch" : "photon_number_basis",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "LiH",
    "geometry": [],
    "symbols": [
      "Li",
      "H"
    ],
    "z-matrix" : mol_str,
  },
    "driver": "energy",
    "model" : {
        "method" : "qed-fci",
        "orbital_basis"  : "sto-3g",
        "photon_basis" : "number_basis",
        "number_of_photon_states" : cavity_dict["number_of_photons"],
        "lambda" : [
            cavity_dict["lambda_vector"][0],
            cavity_dict["lambda_vector"][1],
            cavity_dict["lambda_vector"][2]
        ],
        "omega" : cavity_dict["omega_value"]  
    },
    
    "return_result" : [
        
    ],
    
    "cavity_free_rhf_energy" : 0.,
    "qed_rhf_energy" : 0.,
    "cavity_free_fci_ground_state_energy" : 0.,
}

# function to generate file names based on system details
def generate_file_name(dic, r):
    string_r = "{:.3f}".format(r)
    file_name = dic["molecule"]["molecule_name"] + "_"
    file_name += "r_" + string_r + "_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += str(dictionary["model"]["photon_basis"]) + "_"
    file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
    file_name += "lambda_" + str(dictionary["model"]["lambda"][0]) 
    file_name += "_" + str(dictionary["model"]["lambda"][1]) 
    file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
    file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
    return file_name

In [7]:

print(dictionary)
print(generate_file_name(dictionary, 1.833443))

{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [], 'cavity_free_rhf_energy': 0.0, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': 0.0}
LiH_r_1.833_qed-fci_sto-3g_number_basis_10_lambda_0.0_0.0_0.05_omega_0.12086.json


In [ ]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 50
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))

r_idx = 0
for r in r_array:
    file_name = generate_file_name(dictionary, r) 
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    fci_energy = psi4.energy('fci',ref_wfn=wfn)    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    
    dictionary["return_result"] = [LiH.CIeigs[0], LiH.CIeigs[1], LiH.CIeigs[2], LiH.CIeigs[3], LiH.CIeigs[4], LiH.CIeigs[5], LiH.CIeigs[6]]
    dictionary["cavity_free_rhf_energy"] = scf_e
    dictionary["cavity_free_fci_ground_state_energy"] = fci_energy
    geometry = np.asarray(mol.geometry())
    geometry_list = geometry.tolist()
    dictionary["molecule"]["geometry"] = geometry_list
    E_array[r_idx, :] = LiH.CIeigs[:50]
    r_idx += 1
    print(dictionary)
    json_object = json.dumps(dictionary, indent=4)
    with open(file_name, "w") as outfile:
        outfile.write(json_object)



 Completed QED-RHF in 0.10412216186523438 seconds
 Completed 1HSO Build in 5.793571472167969e-05 seconds
 Completed ERI Build in 0.0029790401458740234 seconds 
 Completed 2D build in 0.0004589557647705078 seconds
 Completed 1G build in 5.698204040527344e-05 seconds
 Completed determinant list in 0.0006430149078369141 seconds 
 Completed constant offset matrix in 0.0008099079132080078 seconds
 Completed Hamiltonian build in 0.2550349235534668 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.33230005115031613], [0.0, 0.0, 2.313316524490643]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': 

 Completed QED-RHF in 0.10610103607177734 seconds
 Completed 1HSO Build in 5.2928924560546875e-05 seconds
 Completed ERI Build in 0.0025720596313476562 seconds 
 Completed 2D build in 0.0004911422729492188 seconds
 Completed 1G build in 5.1975250244140625e-05 seconds
 Completed determinant list in 0.0006079673767089844 seconds 
 Completed constant offset matrix in 0.0004839897155761719 seconds
 Completed Hamiltonian build in 0.265484094619751 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.35942658593809707], [0.0, 0.0, 2.5021586897551855]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result'

 Completed QED-RHF in 0.1053619384765625 seconds
 Completed 1HSO Build in 4.506111145019531e-05 seconds
 Completed ERI Build in 0.0025589466094970703 seconds 
 Completed 2D build in 0.00046181678771972656 seconds
 Completed 1G build in 5.793571472167969e-05 seconds
 Completed determinant list in 0.0006756782531738281 seconds 
 Completed constant offset matrix in 0.00040411949157714844 seconds
 Completed Hamiltonian build in 0.2654399871826172 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.386553120725878], [0.0, 0.0, 2.6910008550197277]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': 

 Completed QED-RHF in 0.10237407684326172 seconds
 Completed 1HSO Build in 4.601478576660156e-05 seconds
 Completed ERI Build in 0.0025169849395751953 seconds 
 Completed 2D build in 0.0004570484161376953 seconds
 Completed 1G build in 5.078315734863281e-05 seconds
 Completed determinant list in 0.0005929470062255859 seconds 
 Completed constant offset matrix in 0.00047516822814941406 seconds
 Completed Hamiltonian build in 0.2636449337005615 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.41367965551365876], [0.0, 0.0, 2.879843020284269]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result':

 Completed QED-RHF in 0.10411477088928223 seconds
 Completed 1HSO Build in 4.6253204345703125e-05 seconds
 Completed ERI Build in 0.002549886703491211 seconds 
 Completed 2D build in 0.0006070137023925781 seconds
 Completed 1G build in 5.6743621826171875e-05 seconds
 Completed determinant list in 0.0006139278411865234 seconds 
 Completed constant offset matrix in 0.0004820823669433594 seconds
 Completed Hamiltonian build in 0.2635059356689453 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4408061903014398], [0.0, 0.0, 3.0686851855488126]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result':

 Completed QED-RHF in 0.10675692558288574 seconds
 Completed 1HSO Build in 4.601478576660156e-05 seconds
 Completed ERI Build in 0.002531290054321289 seconds 
 Completed 2D build in 0.00045180320739746094 seconds
 Completed 1G build in 4.9114227294921875e-05 seconds
 Completed determinant list in 0.0008058547973632812 seconds 
 Completed constant offset matrix in 0.0004360675811767578 seconds
 Completed Hamiltonian build in 0.2600879669189453 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4679327250892207], [0.0, 0.0, 3.2575273508133544]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result':

 Completed QED-RHF in 0.10277485847473145 seconds
 Completed 1HSO Build in 4.6253204345703125e-05 seconds
 Completed ERI Build in 0.0025339126586914062 seconds 
 Completed 2D build in 0.0004379749298095703 seconds
 Completed 1G build in 5.507469177246094e-05 seconds
 Completed determinant list in 0.0006349086761474609 seconds 
 Completed constant offset matrix in 0.0005202293395996094 seconds
 Completed Hamiltonian build in 0.2656998634338379 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'edc918595761fee80a305087184f580b699b21b3'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.49505925987700156], [0.0, 0.0, 3.446369516077897]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result':